Creating a Python function that takes in actual labels and predicted labels and returns these key metrics (Accuracy, Precision, Recall, F1-score, ROC-AUC, and the Confusion Matrix).

In [1]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

def evaluate_model(y_true, y_pred, y_pred_proba=None):

# Initialize dictionary to hold all evaluation metrics
    metrics = {}

    # Calculate and store the evaluation metrics
    metrics['Accuracy'] = accuracy_score(y_true, y_pred)
    metrics['Precision'] = precision_score(y_true, y_pred, zero_division=0)
    metrics['Recall'] = recall_score(y_true, y_pred, zero_division=0)
    metrics['F1-Score'] = f1_score(y_true, y_pred, zero_division=0)

    # Confusion matrix: True Positives, False Positives, True Negatives, False Negatives
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    metrics['Confusion Matrix'] = {
        'True Positives': tp,
        'False Positives': fp,
        'True Negatives': tn,
        'False Negatives': fn
    }
    
    # ROC-AUC Score (only if predicted probabilities are provided)
    if y_pred_proba is not None:
        metrics['ROC-AUC'] = roc_auc_score(y_true, y_pred_proba)
    
    return metrics


Example of using it in practice:

In [2]:
# Assuming `y_test` are actual labels and `y_pred` are predicted labels
# If you have predicted probabilities (for ROC-AUC), include `y_pred_proba` as well

# Example
y_pred = rf_model.predict(X_test_scaled)
y_pred_proba = rf_model.predict_proba(X_test_scaled)[:, 1]  # Probabilities for the positive class (fraud)

# Call the evaluation function
evaluation_results = evaluate_model(y_test, y_pred, y_pred_proba)

# Print results
for metric, value in evaluation_results.items():
    print(f"{metric}: {value}")

NameError: name 'rf_model' is not defined

Visualising Confusion Matrix:

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['Non-Fraud', 'Fraud'],
                yticklabels=['Non-Fraud', 'Fraud'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

# Plot confusion matrix
plot_confusion_matrix(y_test, y_pred)

NameError: name 'y_test' is not defined